In [4]:
import uproot
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.cm as cm


In [5]:
def PtEtaPhiToX(pt,eta,phi):
    return(pt * np.cos(phi))
    
def PtEtaPhiToY(pt,eta,phi):
    return(pt * np.sin(phi))

def PtEtaPhiToZ(pt,eta,phi):
    return(pt * np.sinh(eta))

In [6]:
def XYZToP(px,py,pz):
    return(np.sqrt(px**2 + py**2 + pz**2))
    
def XYZToTheta(px,py,pz):
    return(np.arccos(pz/XYZToP(px,py,pz)))

def XYZToPhi(px,py,pz):
    return(np.arctan(py/px))

In [24]:
def generate_figs(TTree,nblayer_opt):
    #get information of all particles
    #part_px = TTree["Particle/Particle.Px"].array()
    #part_py = TTree["Particle/Particle.Py"].array()
    #part_pz = TTree["Particle/Particle.Pz"].array()

    part_energy = TTree["Particle/Particle.E"].array()
    part_mass = TTree["Particle/Particle.Mass"].array()

    part_status = TTree["Particle.Status"].array()
    
    #extract final state particles
    
    #final_px = part_px[part_status == 1]
    #final_py = part_py[part_status == 1]
    #final_pz = part_pz[part_status == 1]

    final_e = part_energy[part_status == 1]
    final_m = part_mass[part_status == 1]
    
    #baseline cut (nb >= 3, HT>700, njet >= 9)
    #njets = TTree_dijet["Jet"].array()
    jets_btag = TTree["Jet/Jet.BTag"].array()
    #HT = TTree_dijet["ScalarHT/ScalarHT.HT"].array()
    
    nbm = np.sum(jets_btag,axis=1)
    
    
    #convert x, y, z to p, theta, phi
    #final_absP=XYZToP(final_px,final_py,final_pz)
    #final_theta=XYZToTheta(final_px,final_py,final_pz)
    #final_Phi=XYZToPhi(final_px,final_py,final_pz)
    part_theta = TTree["Particle/Particle.Eta"].array()
    final_theta = part_theta[part_status == 1]
    part_Phi = TTree["Particle/Particle.Phi"].array()
    final_Phi = part_theta[part_status == 1]

    
    #get information of all jets

    


    
    #generate figs event by event
    figs = []
    for evt in tqdm(range(1000)):
        bins = (32,32)
        A = np.array(np.array([np.array(final_e[evt]),np.array(final_theta[evt]),np.array(np.array(final_Phi[evt]))]))
        thetas = A[2]
        phis = A[1]
        energies = A[0]   
        center = [np.average(thetas, weights=energies), np.average(phis,weights=energies)]
        range_fig=[[center[0]-np.pi, center[0]+np.pi ],[center[1]-np.pi/2,center[1]+np.pi/2]]
        histo, xedges, yedges =  np.histogram2d(thetas-center[0], phis-center[1], bins=bins, weights=A[0], range=range_fig)
        if (nblayer_opt == 0):
            nblayer = np.eye(32) * nbm[evt]
        elif (nblayer_opt == 1):
            nblayer = np.ones(histo.shape) * nbm[evt]
        elif (nblayer_opt == 2):
            nblayer = np.eye(32, k=3) * nbm[evt]



            
        fig = np.stack((histo, nblayer))
        
        figs.append(fig)
    
        # A: [p,theta,phi]
    
    
    
    return(np.array(figs))

In [25]:
dijet = uproot.open("./jj_inclusive.root")
tttt  = uproot.open("./tttt.root")
ttbar  = uproot.open("./ttbar.root")
ttjj = uproot.open("./ttjj_inclusive.root")

In [26]:
TTree_dijet = dijet["Delphes"]
TTree_tttt  = tttt["Delphes"]
TTree_ttbar  = ttbar["Delphes"]
TTree_ttjj = ttjj["Delphes"]

In [27]:
figs_ttbar_iden = generate_figs(TTree_ttbar, 0)





100%|████████████████| 1000/1000 [00:00<00:00, 1008.49it/s]


In [28]:
figs_ttbar_full = generate_figs(TTree_ttbar, 1)
figs_ttbar_idenk3 = generate_figs(TTree_ttbar, 2)

100%|█████████████████| 1000/1000 [00:01<00:00, 895.61it/s]


In [ ]:
figs_dijet_iden = generate_figs(TTree_dijet, 0)
figs_dijet_full = generate_figs(TTree_dijet, 1)
figs_dijet_idenk3 = generate_figs(TTree_dijet, 2)

In [ ]:
figs_tttt_iden = generate_figs(TTree_tttt, 0)
figs_tttt_full = generate_figs(TTree_tttt, 1)
figs_tttt_idenk3 = generate_figs(TTree_tttt, 2)

In [ ]:
figs_ttjj_iden = generate_figs(TTree_ttjj, 0)
figs_ttjj_full = generate_figs(TTree_ttjj, 1)
figs_ttjj_idenk3 = generate_figs(TTree_ttjj, 2)

In [ ]:
a = int(np.random.rand(1)*1000)
plt.subplot(2,2,1)
plt.imshow(fig_ttjj[a][0], cmap=cm.gray_r)
plt.title("ttjj")
plt.subplot(2,2,2)
plt.imshow(figs_tttt[a][0], cmap=cm.gray_r)
plt.title("tttt")
plt.subplot(2,2,3)
plt.imshow(figs_ttbar[a][0], cmap=cm.gray_r)
plt.title("tt")
plt.subplot(2,2,4)
plt.imshow(figs_dijet[a][0], cmap=cm.gray_r)
plt.title("jj")

In [12]:
#a = int(np.random.rand(1)*100000)
#plt.subplot(1,2,1)
#plt.imshow(figs_dijet[a][0], cmap=cm.gray_r)
#plt.subplot(1,2,2)
#plt.imshow(figs_tttt[a][0], cmap=cm.gray_r)

In [19]:
#np.save("./tttt_nbm",figs_tttt)
#np.save("./jj_nbm",figs_dijet)
#np.save("./ttbar_nbm",figs_ttbar)
#np.save("./ttjj_nbm",fig_ttjj)